<a href="https://colab.research.google.com/github/Zadracker/Basic-Python-Programs/blob/main/RandLA-Net%20Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RandLA-Net Inference on LiDAR Point Cloud (Google Colab)

This notebook runs inference on a `.ply` file using a pre-trained RandLA-Net model. The output will be a segmented `.ply` file that can be visualized in CloudCompare.

## 1. Setup Environment

### Install Dependencies


In [13]:
# Install Python 3.10 and dependencies
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-distutils python3.10-venv -y

# Ensure pip is available for Python 3.10
!python3.10 -m ensurepip

# Upgrade pip, setuptools, and wheel
!python3.10 -m pip install --upgrade pip setuptools wheel

# Update alternatives to make Python 3.10 the default
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --config python3

# Verify the active Python version
!python3 --version

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [14]:
!pip install numpy scipy h5py pandas open3d potpourri3d plyfile trimesh
!pip install torch torchvision torchaudio
!pip install torch-geometric torch-cluster torch-scatter torch-sparse torch-spline-conv
!pip install git+https://github.com/nicolas-chaulet/torch-points3d.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.1/881.1 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 707.0/707.0 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/

### Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Load Model and Data

### Define Paths


In [3]:
CHECKPOINT_PATH = '/content/drive/My Drive/P8T/checkpoint.tar'
INPUT_PLY = '/content/drive/My Drive/P8T/Tibet-2.ply'
OUTPUT_PLY = '/content/drive/My Drive/P8T/Tibet-2-segmented.ply'

### Load Pretrained Model

In [4]:
import torch
from torch_points3d.models.segmentation import RandLANet

# Load model
checkpoint = torch.load(CHECKPOINT_PATH, map_location='cpu')
model = RandLANet()
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

ModuleNotFoundError: No module named 'torch_points3d'


## 3. Run Inference

### Load Point Cloud


In [ ]:
import open3d as o3d
import numpy as np

# Load PLY file
pcd = o3d.io.read_point_cloud(INPUT_PLY)
points = np.asarray(pcd.points)

### Predict Classes



In [ ]:
with torch.no_grad():
    inputs = torch.tensor(points, dtype=torch.float32).unsqueeze(0)
    preds = model(inputs).argmax(dim=2).squeeze().numpy()

### Assign Colors Based on Classes




In [ ]:
import matplotlib.pyplot as plt

NUM_CLASSES = preds.max() + 1
COLORS = plt.get_cmap("tab10")(np.linspace(0, 1, NUM_CLASSES))[:, :3]  # RGB colors
colors = COLORS[preds]

pcd.colors = o3d.utility.Vector3dVector(colors)

### Save Segmented Point Cloud

In [ ]:
o3d.io.write_point_cloud(OUTPUT_PLY, pcd)
print(f"Segmented point cloud saved at: {OUTPUT_PLY}")

## 4. Visualization in CloudCompare

- Download `Tibet-2-segmented.ply` from Google Drive.
- Open it in CloudCompare to view the segmented results.